<a href="https://colab.research.google.com/github/MeinHserhT/CS14115/blob/main/Explore_columns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [149]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [150]:
%cd drive/MyDrive/'gStore Revenue Prediction'/data

[Errno 2] No such file or directory: 'drive/MyDrive/gStore Revenue Prediction/data'
/content/drive/.shortcut-targets-by-id/17_NpRqJ8sX5090M59mFJlt5fuEMyTmdS/gStore Revenue Prediction/data


In [151]:
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
import os
import json
from pandas import json_normalize
from scipy.stats import norm
import datetime
import ast

import gc
gc.enable()

All columns description here: 
- https://brandee.edu.vn/glossary/3437719-analytics-en/
- https://support.google.com/analytics/answer/3437719?hl=vi 

# EXPLORE ALL COLUMNS

## 1. fullVisitorId
A unique identifier for each user of the Google Merchandise Store.

In [152]:
df = pd.read_csv('fullVisitorId.csv', names = ['index', 'fullVisitorId']).set_index('index')
df

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,fullVisitorId
index,
0,3162355547410993243
1,8934116514970143966
2,7992466427990357681
3,9075655783635761930
4,6960673291025684308
...,...
1708332,5123779100307500332
1708333,7231728964973959842
1708334,5744576632396406899


In [153]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,fullVisitorId,value_counts
0,1957458976293878100,338
1,7282998257608986241,265
2,3884810646891698298,221
3,0824839726118485274,199
4,7477638593794484792,187
...,...,...
1367966,22186703153482556,1
1367967,2218649836394656223,1
1367968,2218645366307164399,1
1367969,2218644562920332378,1


## 2. ChannelGrouping
Channel Groupings are rule-based groupings of your traffic sources

In [154]:
df = pd.read_csv('channelGrouping.csv', names = ['index', 'channelGrouping']).set_index('index')
df.head()

,channelGrouping
index,
0,Organic Search
1,Referral
2,Direct
3,Organic Search
4,Organic Search


In [155]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,channelGrouping,value_counts
0,Organic Search,738963
1,Social,354971
2,Direct,273134
3,Referral,211307
4,Display,51283
5,Paid Search,45627
6,Affiliates,32915
7,(Other),137


In [156]:
# based on: https://dashthis.com/blog/google-analytics-display-traffic/
des = {
"Organic Search": "This traffic found your site in a search engine such as Google or Bing. If you’re focusing on optimizing pages for search engines, this is an important channel to watch;",
"Display": "This traffic found your site by clicking on an ad that you ran on another website. Banner ads on blogs and image ads on news sites are some common generators of display traffic;",
"Direct": "This traffic came to your site by entering your URL directly into the address bar of browsers. Keep an eye on this one if you've been running offline or traditional media ads like print, TV, or radio, because they require audiences to remember and type out your web address;",
"Referral": "This traffic followed a backlink from another website to yours, and you'll see this traffic if it doesn't fall under one of the other buckets;",
"Paid Search": "this traffic comes from your paid search ads which appear in the search results of Bing, Google, or other search network players like AOL and Ask.com;",
"Social": "This traffic will be counted from people who find your page through an associated social media account. Check in on users who are landing on your page as a result of social media accounts like Facebook, LinkedIn, or Twitter;",
"Email": "This traffic clicked on links from email campaigns, follow up emails, and even email signatures;",
"(Other)": "If GA greets your web traffic with a shrug emoji, they'll throw it in this channel. Note that there are often better ways to group this traffic.",
}
des_df = pd.DataFrame(list(des.items()), columns = ['channelGrouping', 'description'])

In [157]:
pd.merge(vl_count_df, des_df, on=['channelGrouping']).set_index('channelGrouping')

,value_counts,description
channelGrouping,,
Organic Search,738963,"This traffic found your site in a search engine such as Google or Bing. If you’re focusing on optimizing pages for search engines, this is an important channel to watch;"
Social,354971,"This traffic will be counted from people who find your page through an associated social media account. Check in on users who are landing on your page as a result of social media accounts like Facebook, LinkedIn, or Twitter;"
Direct,273134,"This traffic came to your site by entering your URL directly into the address bar of browsers. Keep an eye on this one if you've been running offline or traditional media ads like print, TV, or radio, because they require audiences to remember and type out your web address;"
Referral,211307,"This traffic followed a backlink from another website to yours, and you'll see this traffic if it doesn't fall under one of the other buckets;"
Display,51283,This traffic found your site by clicking on an ad that you ran on another website. Banner ads on blogs and image ads on news sites are some common generators of display traffic;
Paid Search,45627,"this traffic comes from your paid search ads which appear in the search results of Bing, Google, or other search network players like AOL and Ask.com;"
(Other),137,"If GA greets your web traffic with a shrug emoji, they'll throw it in this channel. Note that there are often better ways to group this traffic."


## 3. Date (format into Day-Month-Year)
The date on which the user visited the Store.

In [158]:
col_name = 'date'
df = pd.read_csv(col_name + '.csv', names = ['index', col_name]).set_index('index')
df

,date
index,
0,20171016
1,20171016
2,20171016
3,20171016
4,20171016
...,...
1708332,20170104
1708333,20170104
1708334,20170104


In [159]:
# todo day-month-year
df['date'] = df['date'].apply(lambda x: str(x)[:4] + ' ' + str(x)[4:6] + ' ' + str(x)[6:])
df[['year', 'month', 'day']] = df['date'].str.split(' ', expand=True)
df.drop(columns=['date']).sort_values(by=['year', 'month', 'day'])

,year,month,day
index,,,
995708,2016,08,01
995709,2016,08,01
995710,2016,08,01
995711,2016,08,01
995712,2016,08,01
...,...,...,...
1032486,2018,04,30
1032487,2018,04,30
1032488,2018,04,30


## 4. Device
The specifications for the device used to access the Store.

### 4.1. device.browser
The browser used (e.g., "Chrome" or "Firefox")

In [160]:
df = pd.read_csv('device.browser.csv', names = ['index', 'device.browser']).set_index('index')
df

,device.browser
index,
0,Firefox
1,Chrome
2,Chrome
3,Chrome
4,Chrome
...,...
1708332,Chrome
1708333,Chrome
1708334,Android Webview


In [161]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.browser,value_counts
0,Chrome,1173056
1,Safari,312165
2,Firefox,63845
3,Internet Explorer,35474
4,Android Webview,34266
...,...,...
124,DoCoMo,1
125,Dillo,1
126,DDG-Android-3.1.1,1
127,Changa 99695759,1


In [162]:
# temp_df = vl_count_df.apply(lambda x: x if x['value_counts'] > 941 else {'device.browser': 'Other', 'value_counts': x['value_counts']}, axis = 1)
# temp_df['percent'] = temp_df['value_counts'] / 1708337 * 100
# temp_df.groupby('device.browser').sum().sort_values('value_counts')

In [163]:
vl_count_df.groupby('value_counts')['device.browser'].apply(list).to_frame()

,device.browser
value_counts,
1,"[ejpxuidzlmagvthsfbqnkwyocr, efkaxnbyohqtspzlvcwrjmigdu, eosutpkiahjzvdgcwxlmyfqbrn, epxmjusghnvircdfkwqlotzbay, ecwozghsufybtdkjrlvxpamiqn, ecgiwapzltrkujdhmqsbxfonvy, flobzsdixhuwqakptjmcrveygn, jdbknvrluyeaxoipgwczmthsqf, flwadqukonrjegpbisyxztvhcm, rpfanjzoxyemsgbtichqkudwlv, ymzsbiduaejrchvxlwkfnqgtop, wvsmagudcqeytijorlhxnfzkbp, wncrmxukofqljsgvzahiybpdet, wfpknuqxovyilmrdzbhgtecjas, wdhtapevfnqzskcroxgjmiybul, vjebamzrktwcysxpdlonhiufqg, uybjlgntzwpacihremkqsxdovf, ujvrzsonxihlgaqdmkwtbfcpey, uhdypcxbgzajmeqwlofnrsitkv, tfowdqmibyshaklxuregpcnzvj, subjectAgent: NoticiasBoom, starmaker, rbydojcflwzvnuaepmsgxhiktq, fspmihbxzowgnuctrqykjlvade, ohukwejvqmdtibfrzpycgxanls, ohfgqlpiuyknvmbctszjarxdwe, njroiedbwpmvykqlatxzuhcfgs, mhwxofpevcagujznbsiqlrkytd, lxjwoyfivgdbkqtuzsrmhencpa, lpmqaxwbzyteokrfusnjhvdigc, lhkbrtuwomdeafnqygvxcspizj, kqebrzuwmiycxdvtoljnhsfpga, jscatcher, dkagwlhmfqxercuozpnbvtsiyj, ighfsbrmpoctzjqxlywdenvuka, hbijxvdyrgnatwzmlcpkfusqoe, dohyinzpvbsktjeguxmrqcwafl, NokiaC7-00, cnwmpegudakrqzljtvfxohbysi, cajrnbtvqwfkolzyxushpdgime, ;__CT_JOB_ID__:a80e8e16-6e98-455b-885a-a4dd40f3d344;, ;__CT_JOB_ID__:a7ed0808-e70c-4b19-b1a3-1018bbb7dc7f;, ;__CT_JOB_ID__:a4f837b8-8d78-4c42-ba9a-d870cf1a4a7e;, ;__CT_JOB_ID__:a24a8978-e5e8-4dc9-af66-c4ed89ea25d7;, ;__CT_JOB_ID__:97909e28-4228-4b55-8ad5-cc791f2b583c;, ;__CT_JOB_ID__:89e59554-ad41-4e94-957b-f12bd012530c;, ;__CT_JOB_ID__:85da5736-a78e-45a9-837e-f5a53e5cd725;, ;__CT_JOB_ID__:7e575295-571e-4e82-9254-7f2c8bbb9183;, ;__CT_JOB_ID__:76fd1acb-e365-43c0-b967-908bcf5d5b59;, ;__CT_JOB_ID__:6e9dcf2f-f58f-4938-91e3-77e00868177b;, ;__CT_JOB_ID__:65da7e5f-0f05-4b5d-8d31-1f4d470a2b82;, ;__CT_JOB_ID__:58e2ecba-7666-4a10-b498-8216457ce472;, ;__CT_JOB_ID__:4333777f-bb0c-4a18-935e-df5658dbce2d;, ;__CT_JOB_ID__:2e0eca60-83ab-482d-bb81-343d113254fb;, ;__CT_JOB_ID__:2547db0b-ec43-452a-a0d4-ff42b7dc7907;, ;__CT_JOB_ID__:0b39e7ca-1431-42e3-ba1f-9d8951a65840;, ;__CT_JOB_ID__:0a075729-93a5-43d0-9638-4cbd41d5f5a5;, ;__CT_JOB_ID__:d14534ff-e2fc-4692-92aa-e34508f1c418;, ;__CT_JOB_ID__:dd6177aa-1baa-4007-9b38-b7cab4f7611c;, ;__CT_JOB_ID__:fe02e46f-b6ae-41f1-8563-3b40bbb623a9;, M5, bsfnwveckhgpdoyjxmizruqtla, ajsqixbltuvwpmdcokfyzhgren, afjurnqyolshpibxczdwktmvge, [Use default User-agent string] LIVRENPOCHE, User Agent, TCL P500M, Reddit, KINGSUN-F4, ADM, IE with Chrome Frame, Hisense M20-M_LTE, HTC802t_TD, DoCoMo, Dillo, DDG-Android-3.1.1, Changa 99695759, zurcqesbhljxmpwdgnvkoyafit]"
2,"[NokiaE52-1, Amazon.com, Android Runtime]"
3,"[MQQBrowser, LYF_LS_4002_11, ThumbSniper, SAMSUNG-SM-B355E Opera]"
4,"[no-ua, CSM Click, Autn-WKOOP, Netscape, DASH_JR_3G]"
5,"[DESKTOP, Browser]"
6,"[Konqueror, YE]"
7,"[Nichrome, 0]"
8,"[+Simple Browser, Lunascape]"
12,[(not set)]


### 4.2 device.browserSize (*drop this column*)

The browser size in lenght and width.

In [164]:
df = pd.read_csv('device.browserSize.csv', names = ['index', 'device.browserSize']).set_index('index')
df

,device.browserSize
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [165]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.browserSize,value_counts
0,not available in demo dataset,1708337


### 4.3 device.browserVersion (*drop this column*)

In [166]:
df = pd.read_csv('device.browserVersion.csv', names = ['index', 'device.browserVersion']).set_index('index')
df

,device.browserVersion
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [167]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.browserVersion,value_counts
0,not available in demo dataset,1708337


### 4.4 device.deviceCategory
The type of device (Mobile, Tablet, Desktop).

In [168]:
df = pd.read_csv('device.deviceCategory.csv', names = ['index', 'device.deviceCategory']).set_index('index')
df

,device.deviceCategory
index,
0,desktop
1,desktop
2,mobile
3,desktop
4,desktop
...,...
1708332,desktop
1708333,mobile
1708334,mobile


In [169]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.deviceCategory,value_counts
0,desktop,1171579
1,mobile,471336
2,tablet,65422


### 4.5 device.mobileDeviceInfo (*drop this column*)
The branding, model, and marketing name used to identify the mobile device.

In [170]:
df = pd.read_csv('device.mobileDeviceInfo.csv', names = ['index', 'device.mobileDeviceInfo']).set_index('index')
df

,device.mobileDeviceInfo
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [171]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.mobileDeviceInfo,value_counts
0,not available in demo dataset,1708337


### 4.6 device.mobileDeviceMarketingName (*drop this column*)

In [172]:
df = pd.read_csv('device.mobileDeviceMarketingName.csv', names = ['index', 'device.mobileDeviceMarketingName']).set_index('index')
df

,device.mobileDeviceMarketingName
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [173]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.mobileDeviceMarketingName,value_counts
0,not available in demo dataset,1708337


### 4.7 device.mobileDeviceModel (*drop this column*)

In [174]:
df = pd.read_csv('device.mobileDeviceModel.csv', names = ['index', 'device.mobileDeviceModel']).set_index('index')
df

,device.mobileDeviceModel
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [175]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.mobileDeviceModel,value_counts
0,not available in demo dataset,1708337


### 4.8 device.mobileInputSelector (*drop this column*)
Selector (e.g., touchscreen, joystick, clickwheel, stylus) used on the mobile device.

In [176]:
df = pd.read_csv('device.mobileInputSelector.csv', names = ['index', 'device.mobileInputSelector']).set_index('index')
df

,device.mobileInputSelector
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [177]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.mobileInputSelector,value_counts
0,not available in demo dataset,1708337


### 4.9 device.mobileDeviceMarketingName (*drop this column*)

In [178]:
df = pd.read_csv('device.mobileDeviceMarketingName.csv', names = ['index', 'device.mobileDeviceMarketingName']).set_index('index')
df

,device.mobileDeviceMarketingName
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [179]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.mobileDeviceMarketingName,value_counts
0,not available in demo dataset,1708337


### 4.10 device.operatingSystem

In [180]:
df = pd.read_csv('device.operatingSystem.csv', names = ['index', 'device.operatingSystem']).set_index('index')
df

,device.operatingSystem
index,
0,Windows
1,Chrome OS
2,Android
3,Windows
4,Windows
...,...
1708332,Windows
1708333,Android
1708334,Android


In [181]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.operatingSystem,value_counts
0,Windows,619720
1,Macintosh,438514
2,Android,299386
3,iOS,219334
4,Linux,63971
5,Chrome OS,51318
6,(not set),11815
7,Windows Phone,1675
8,Samsung,911
9,Tizen,709


### 4.11 device.operatingSystemVersion (*drop this column*)

In [182]:
df = pd.read_csv('device.operatingSystemVersion.csv', names = ['index', 'device.operatingSystemVersion']).set_index('index')
df

,device.operatingSystemVersion
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [183]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.operatingSystemVersion,value_counts
0,not available in demo dataset,1708337


### 4.12 device.isMobile

In [184]:
df = pd.read_csv('device.isMobile.csv', names = ['index', 'device.isMobile']).set_index('index')
df

,device.isMobile
index,
0,False
1,False
2,True
3,False
4,False
...,...
1708332,False
1708333,True
1708334,True


In [185]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.isMobile,value_counts
0,False,1171670
1,True,536667


### 4.13 device.mobileDeviceBranding (*drop this column*)

In [186]:
df = pd.read_csv('device.mobileDeviceBranding.csv', names = ['index', 'device.mobileDeviceBranding']).set_index('index')
df

,device.mobileDeviceBranding
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [187]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.mobileDeviceBranding,value_counts
0,not available in demo dataset,1708337


### 4.14 device.flashVersion (*drop this column*)
The version of the Adobe Flash plugin that is installed on the browser.

In [188]:
df = pd.read_csv('device.flashVersion.csv', names = ['index', 'device.flashVersion']).set_index('index')
df

,device.flashVersion
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [189]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.flashVersion,value_counts
0,not available in demo dataset,1708337


### 4.15 device.language (*drop this column*)

In [190]:
df = pd.read_csv('device.language.csv', names = ['index', 'device.language']).set_index('index')
df

,device.language
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [191]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.language,value_counts
0,not available in demo dataset,1708337


### 4.16 device.screenColors (*drop this column*)
Number of colors supported by the display, expressed as the bit-depth (e.g., "8-bit", "24-bit", etc.).

In [192]:
df = pd.read_csv('device.screenColors.csv', names = ['index', 'device.screenColors']).set_index('index')
df

,device.screenColors
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [193]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.screenColors,value_counts
0,not available in demo dataset,1708337


### 4.17 device.screenResolution (*drop this column*)

In [194]:
df = pd.read_csv('device.screenResolution.csv', names = ['index', 'device.screenResolution']).set_index('index')
df

,device.screenResolution
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [195]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,device.screenResolution,value_counts
0,not available in demo dataset,1708337


## 5. GeoNetwork
This section contains information about the geography of the user.

### 5.1 geoNetwork.continent

In [196]:
df = pd.read_csv('geoNetwork.continent.csv', names = ['index', 'geoNetwork.continent']).set_index('index')
df

,geoNetwork.continent
index,
0,Europe
1,Americas
2,Americas
3,Asia
4,Americas
...,...
1708332,Americas
1708333,Asia
1708334,Asia


In [197]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,geoNetwork.continent,value_counts
0,Americas,877403
1,Asia,396719
2,Europe,368037
3,Africa,35481
4,Oceania,28180
5,(not set),2517


### 5.2 geoNetwork.subContinent	

In [198]:
df = pd.read_csv('geoNetwork.subContinent.csv', names = ['index', 'geoNetwork.subContinent']).set_index('index')
df

,geoNetwork.subContinent
index,
0,Western Europe
1,Northern America
2,Northern America
3,Western Asia
4,Central America
...,...
1708332,Caribbean
1708333,Southern Asia
1708334,Eastern Asia


In [199]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,geoNetwork.subContinent,value_counts
0,Northern America,768345
1,Southeast Asia,121634
2,Southern Asia,121062
3,Western Europe,115153
4,Northern Europe,111693
5,Eastern Asia,91072
6,South America,75112
7,Eastern Europe,74007
8,Southern Europe,67184
9,Western Asia,60966


### 5.3 geoNetwork.country

In [200]:
df = pd.read_csv('geoNetwork.country.csv', names = ['index', 'geoNetwork.country']).set_index('index')
df

,geoNetwork.country
index,
0,Germany
1,United States
2,United States
3,Turkey
4,Mexico
...,...
1708332,Puerto Rico
1708333,Sri Lanka
1708334,South Korea


In [201]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,geoNetwork.country,value_counts
0,United States,717217
1,India,105317
2,United Kingdom,73341
3,Canada,51057
4,Germany,38516
...,...,...
223,St. Pierre & Miquelon,1
224,Norfolk Island,1
225,Solomon Islands,1
226,Montserrat,1


In [202]:
vl_count_df.groupby('value_counts')['geoNetwork.country'].apply(list).to_frame()

,geoNetwork.country
value_counts,
1,"[Anguilla, Micronesia, Eritrea, Samoa, São Tomé & Príncipe, St. Helena, Tonga, St. Pierre & Miquelon, Norfolk Island, Solomon Islands, Montserrat, Åland Islands]"
2,"[St. Barthélemy, American Samoa, Marshall Islands]"
3,[Cook Islands]
4,"[Comoros, Dominica, Seychelles, Vanuatu]"
5,"[Equatorial Guinea, St. Martin]"
...,...
38516,[Germany]
51057,[Canada]
73341,[United Kingdom]


### 5.4 geoNetwork.region
The region from which sessions originate, derived from IP addresses. In the U.S., a region is a state, such as New York.

In [203]:
df = pd.read_csv('geoNetwork.region.csv', names = ['index', 'geoNetwork.region']).set_index('index')
df

,geoNetwork.region
index,
0,not available in demo dataset
1,California
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,Seoul


In [204]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 *100
vl_count_df

,geoNetwork.region,value_counts,percent
0,not available in demo dataset,932959,54.612117
1,California,206669,12.097672
2,(not set),49774,2.913594
3,New York,49733,2.911194
4,England,25824,1.511646
...,...,...,...
478,Maha Sarakham,6,0.000351
479,Braga,6,0.000351
480,Binh Phuoc,6,0.000351
481,Abruzzo,6,0.000351


### 5.5 geoNetwork.metro

src: https://www.thebalancecareers.com/what-is-a-designated-market-area-dma-2315180

The Designated Market Area (DMA) from which sessions originate.

A designated market area (DMA) is a geographic region where Nielsen, the ratings company, analyzes and quantifies how television is viewed.

In [205]:
df = pd.read_csv('geoNetwork.metro.csv', names = ['index', 'geoNetwork.metro']).set_index('index')
df

,geoNetwork.metro
index,
0,not available in demo dataset
1,San Francisco-Oakland-San Jose CA
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,(not set)


In [206]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 *100
vl_count_df

,geoNetwork.metro,value_counts,percent
0,not available in demo dataset,932959,54.612117
1,(not set),386896,22.647522
2,San Francisco-Oakland-San Jose CA,182745,10.697245
3,New York NY,50419,2.951350
4,London,23643,1.383978
...,...,...,...
118,Tallahassee FL-Thomasville GA,6,0.000351
119,Dayton OH,6,0.000351
120,Des Moines-Ames IA,6,0.000351
121,Springfield-Holyoke MA,6,0.000351


### 5.6 geoNetwork.city

In [207]:
df = pd.read_csv('geoNetwork.city.csv', names = ['index', 'geoNetwork.city']).set_index('index')
df

,geoNetwork.city
index,
0,not available in demo dataset
1,Cupertino
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,Seoul


In [208]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 *100
vl_count_df

,geoNetwork.city,value_counts,percent
0,not available in demo dataset,932959,54.612117
1,Mountain View,74110,4.338137
2,(not set),65867,3.855621
3,New York,49460,2.895213
4,San Francisco,36960,2.163508
...,...,...,...
951,Daly City,4,0.000234
952,Morgan Hill,4,0.000234
953,North Creek,4,0.000234
954,Boise,3,0.000176


### 5.7 geoNetwork.cityId (*drop this column*)

In [209]:
df = pd.read_csv('geoNetwork.cityId.csv', names = ['index', 'geoNetwork.cityId']).set_index('index')
df

,geoNetwork.cityId
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [210]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,geoNetwork.cityId,value_counts
0,not available in demo dataset,1708337


### 5.8 geoNetwork.latitude (*drop this column*)
The approximate latitude of users' city, derived from their IP addresses or Geographical IDs. Locations north of the equator have positive latitudes and locations south of the equator have negative latitudes.

In [211]:
df = pd.read_csv('geoNetwork.latitude.csv', names = ['index', 'geoNetwork.latitude']).set_index('index')
df

,geoNetwork.latitude
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [212]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,geoNetwork.latitude,value_counts
0,not available in demo dataset,1708337


### 5.9 geoNetwork.longitude (*drop this column*)
The approximate longitude of users' city, derived from their IP addresses or Geographical IDs. Locations east of the prime meridian have positive longitudes and locations west of the prime meridian have negative longitudes.


In [213]:
df = pd.read_csv('geoNetwork.longitude.csv', names = ['index', 'geoNetwork.longitude']).set_index('index')
df

,geoNetwork.longitude
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [214]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,geoNetwork.longitude,value_counts
0,not available in demo dataset,1708337


### 5.10 geoNetwork.networkDomain
The domain name of user's ISP, derived from the domain name registered to the ISP's IP address.



In [215]:
df = pd.read_csv('geoNetwork.networkDomain.csv', names = ['index', 'geoNetwork.networkDomain']).set_index('index')
df

,geoNetwork.networkDomain
index,
0,(not set)
1,(not set)
2,windjammercable.net
3,unknown.unknown
4,prod-infinitum.com.mx
...,...
1708332,prtc.net
1708333,unknown.unknown
1708334,unknown.unknown


In [216]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 *10
vl_count_df

,geoNetwork.networkDomain,value_counts,percent
0,(not set),499049,2.921256
1,unknown.unknown,269796,1.579290
2,comcast.net,55486,0.324795
3,rr.com,28715,0.168087
4,verizon.net,26547,0.155397
...,...,...,...
41977,hochland.com,1,0.000006
41978,hochrheinnet.de,1,0.000006
41979,omygods.com,1,0.000006
41980,hockaday.org,1,0.000006


### 5.11 geoNetwork.networkLocation (*drop this column*)
The names of the service providers used to reach the property. For example, if most users of the website come via the major cable internet service providers, its value will be these service providers' names.

In [217]:
df = pd.read_csv('geoNetwork.networkLocation.csv', names = ['index', 'geoNetwork.networkLocation']).set_index('index')
df

,geoNetwork.networkLocation
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [218]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,geoNetwork.networkLocation,value_counts
0,not available in demo dataset,1708337


## 6. SocialEngagementType 
Engagement type, either "Socially Engaged" or "Not Socially Engaged".

In [219]:
df = pd.read_csv('socialEngagementType.csv', names = ['index', 'socialEngagementType']).set_index('index')
df.head()

,socialEngagementType
index,
0,Not Socially Engaged
1,Not Socially Engaged
2,Not Socially Engaged
3,Not Socially Engaged
4,Not Socially Engaged


In [220]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,socialEngagementType,value_counts
0,Not Socially Engaged,1708337


## 7. Totals 
This section contains aggregate values across the session.

### 7.1 totals.bounces
Total bounces (for convenience). For a bounced session, the value is 1, otherwise it is null. 
> When a user opens a single page on your site and then exits without triggering any other requests to the Analytics server during that session.

In [221]:
df = pd.read_csv('totals.bounces.csv', names = ['index', 'totals.bounces']).set_index('index')
df.head()

,totals.bounces
index,
0,1.0
1,NaN
2,NaN
3,NaN
4,NaN


In [222]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,totals.bounces,value_counts
0,1.0,871578
1,NaN,836759


### 7.2 totals.hits
Total number of hits within the session.

In [223]:
df = pd.read_csv('totals.hits.csv', names = ['index', 'totals.hits']).set_index('index')
df.head()

,totals.hits
index,
0,1
1,2
2,2
3,2
4,2


In [224]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,totals.hits,value_counts
0,1,864064
1,2,237499
2,3,134435
3,4,80875
4,5,63687
...,...,...
292,262,1
293,260,1
294,259,1
295,204,1


### 7.3 totals.newVisits
Total number of new users in session (for convenience). If this is the first visit, this value is 1, otherwise it is null.

In [225]:
df = pd.read_csv('totals.newVisits.csv', names = ['index', 'totals.newVisits']).set_index('index')
df.head()

,totals.newVisits
index,
0,1.0
1,NaN
2,1.0
3,1.0
4,1.0


In [226]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,totals.newVisits,value_counts
0,1.0,1307430
1,NaN,400907


### 7.4 totals.pageviews
Total number of pageviews within the session.

In [227]:
df = pd.read_csv('totals.pageviews.csv', names = ['index', 'totals.pageviews']).set_index('index')
df.head()

,totals.pageviews
index,
0,1.0
1,2.0
2,2.0
3,2.0
4,2.0


In [228]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,totals.pageviews,value_counts
0,1.0,876328
1,2.0,249794
2,3.0,142896
3,4.0,86666
4,5.0,64712
...,...,...
226,213.0,1
227,215.0,1
228,219.0,1
229,220.0,1


### 7.5 totals.sessionQualityDim
An estimate of how close a particular session was to transacting, ranging from 1 to 100, calculated for each session. A value closer to 1 indicates a low session quality, or far from transacting, while a value closer to 100 indicates a high session quality, or very close to transacting. A value of 0 indicates that Session Quality is not calculated for the selected time range.

In [229]:
df = pd.read_csv('totals.sessionQualityDim.csv', names = ['index', 'totals.sessionQualityDim']).set_index('index')
df.head()

,totals.sessionQualityDim
index,
0,1.0
1,2.0
2,1.0
3,1.0
4,1.0


In [230]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,totals.sessionQualityDim,value_counts,percent
0,NaN,835274,48.893983
1,1.0,717560,42.003422
2,2.0,52517,3.074159
3,3.0,17066,0.998983
4,4.0,9552,0.559140
...,...,...,...
96,96.0,52,0.003044
97,97.0,27,0.001580
98,98.0,9,0.000527
99,99.0,5,0.000293


### 7.6 totals.timeOnSite
The total time on screen in seconds.


In [231]:
df = pd.read_csv('totals.timeOnSite.csv', names = ['index', 'totals.timeOnSite']).set_index('index')
df.head()

,totals.timeOnSite
index,
0,NaN
1,28.0
2,38.0
3,1.0
4,52.0


In [232]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,totals.timeOnSite,value_counts,percent
0,NaN,874294,51.178076
1,5.0,9862,0.577287
2,4.0,9738,0.570028
3,6.0,9150,0.535609
4,7.0,8221,0.481228
...,...,...,...
4770,4356.0,1,0.000059
4771,3452.0,1,0.000059
4772,3243.0,1,0.000059
4773,4364.0,1,0.000059


### 7.7 totals.totalTransactionRevenue
Total transaction revenue, expressed as the value passed to Analytics multiplied by 10^6 (e.g., 2.40 would be given as 2400000).

In [233]:
df = pd.read_csv('totals.totalTransactionRevenue.csv', names = ['index', 'totals.totalTransactionRevenue']).set_index('index')
df.head()

,totals.totalTransactionRevenue
index,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [234]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,totals.totalTransactionRevenue,value_counts,percent
0,NaN,1689823,98.916256
1,24990000.0,147,0.008605
2,23990000.0,143,0.008371
3,22990000.0,134,0.007844
4,25990000.0,120,0.007024
...,...,...,...
8502,105850000.0,1,0.000059
8503,105790000.0,1,0.000059
8504,105770000.0,1,0.000059
8505,105760000.0,1,0.000059


### 7.8 totals.transactions
Total number of ecommerce transactions within the session.


In [235]:
df = pd.read_csv('totals.transactions.csv', names = ['index', 'totals.transactions']).set_index('index')
df.head()

,totals.transactions
index,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN


In [236]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,totals.transactions,value_counts,percent
0,NaN,1689778,98.913622
1,1.0,18048,1.056466
2,2.0,420,0.024585
3,3.0,43,0.002517
4,4.0,17,0.000995
5,5.0,11,0.000644
6,6.0,6,0.000351
7,7.0,5,0.000293
8,8.0,3,0.000176
9,12.0,2,0.000117


### 7.9 totals.visits
The number of sessions (for convenience). This value is 1 for sessions with interaction events. The value is null if there are no interaction events in the session.


In [237]:
df = pd.read_csv('totals.visits.csv', names = ['index', 'totals.visits']).set_index('index')
df.head()

,totals.visits
index,
0,1
1,1
2,1
3,1
4,1


In [238]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,totals.visits,value_counts
0,1,1708337


## 8. Traffic Source 
This section contains information about the Traffic Source from which the session originated.

### 8.1 trafficSource.adContent
The ad content of the traffic source. Can be set by the utm_content URL parameter.


In [239]:
df = pd.read_csv('trafficSource.adContent.csv', names = ['index', 'trafficSource.adContent']).set_index('index')
df

,trafficSource.adContent
index,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1708332,NaN
1708333,NaN
1708334,NaN


In [240]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.adContent,value_counts,percent
0,NaN,1643600,96.210525
1,Google Merchandise Store,39566,2.316054
2,Google Merchandise Collection,6762,0.395824
3,Placement Accessores 300 x 250,3040,0.177951
4,Smart display ad - 8/17/2017,2664,0.155941
...,...,...,...
72,Google store,2,0.000117
73,Swag w/ Google Logos,1,0.000059
74,Google Apparel,1,0.000059
75,GA Help Center,1,0.000059


In [241]:
vl_count_df.groupby('value_counts')['trafficSource.adContent'].apply(list).to_frame()

,trafficSource.adContent
value_counts,
1,"[Swag w/ Google Logos, Google Apparel, GA Help Center, Men's Apparel from Google]"
2,"[google store, cool, Free Shipping!, Google store]"
3,"[Men's-Outerwear Google Apparel, Ad from 2/17/17, url_builder, Full auto ad NATIVE ONLY, free shipping, Full auto ad with Primary Color]"
4,[Official Google Merchandise - Fast Shipping]
5,[{KeyWord:Google Branded Outerwear}]
6,[Drinkware 120x600]
7,"[Full auto ad TEXT/NATIVE, Office 2018 - 120 x 600, visit us again]"
8,"[Google Paraphernalia, Google Store, test_tyler_hr_merchant]"
9,[Want Google Sunglasses]


### 8.2 trafficSource.adwordsClickInfo.adNetworkType
Network Type. Takes one of the following values: 
- Google Search
- Content
- Search partners
- Ad Exchange
- Yahoo Japan Search
- Yahoo Japan AFS
- unknown


In [242]:
df = pd.read_csv('trafficSource.adwordsClickInfo.adNetworkType.csv', names = ['index', 'trafficSource.adwordsClickInfo.adNetworkType']).set_index('index')
df

,trafficSource.adwordsClickInfo.adNetworkType
index,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1708332,NaN
1708333,NaN
1708334,NaN


In [243]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.adwordsClickInfo.adNetworkType,value_counts,percent
0,NaN,1633063,95.593727
1,Content,42223,2.471585
2,Google Search,33043,1.934220
3,Search partners,8,0.000468


### 8.3 trafficSource.adworsClickInfo.criteriaParameters (*drop this column*)

Descriptive string for the targeting criterion.


In [244]:
df = pd.read_csv('trafficSource.adwordsClickInfo.criteriaParameters.csv', names = ['index', 'trafficSource.adwordsClickInfo.criteriaParameters']).set_index('index')
df

,trafficSource.adwordsClickInfo.criteriaParameters
index,
0,not available in demo dataset
1,not available in demo dataset
2,not available in demo dataset
3,not available in demo dataset
4,not available in demo dataset
...,...
1708332,not available in demo dataset
1708333,not available in demo dataset
1708334,not available in demo dataset


In [245]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,trafficSource.adwordsClickInfo.criteriaParameters,value_counts
0,not available in demo dataset,1708337


### 8.4 trafficSource.adwordsClickInfo.gclId (*drop this column*)
The Google Click ID.


In [246]:
df = pd.read_csv('trafficSource.adwordsClickInfo.gclId.csv', names = ['index', 'trafficSource.adwordsClickInfo.gclId']).set_index('index')
df

,trafficSource.adwordsClickInfo.gclId
index,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1708332,NaN
1708333,NaN
1708334,NaN


In [247]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.adwordsClickInfo.gclId,value_counts,percent
0,NaN,1632914,95.585005
1,CN_Whvvc_9UCFd6LswodGTgKCQ,74,0.004332
2,Cj0KEQjwmIrJBRCRmJ_x7KDo-9oBEiQAuUPKMufMpuG3ZdwYO8GTsjiBFd5MPHStZa9y_9NCrI8X97oaAglc8P8HAQ,70,0.004098
3,COT1-vPT4tYCFZWNswodcwsHxg,60,0.003512
4,CN3fusbjvtYCFQsmhgodIEQO-g,51,0.002985
...,...,...,...
59004,CNrQoY3G5dYCFYY1aQodYqkIow,1,0.000059
59005,CNrQvuWA29YCFYWRjwodjRYHFA,1,0.000059
59006,CNrSoKH5ns8CFdgegQodafIMpg,1,0.000059
59007,CNrSp6-AyNYCFctXDQodnQEOOg,1,0.000059


### 8.5 trafficSource.adwordsClickInfo.isVideoAd
True if it is a Trueview video ad.


In [248]:
df = pd.read_csv('trafficSource.adwordsClickInfo.isVideoAd.csv', names = ['index', 'trafficSource.adwordsClickInfo.isVideoAd']).set_index('index')
df

,trafficSource.adwordsClickInfo.isVideoAd
index,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1708332,NaN
1708333,NaN
1708334,NaN


In [249]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.adwordsClickInfo.isVideoAd,value_counts,percent
0,NaN,1633063,95.593727
1,False,75274,4.406273


### 8.6 trafficSource.adwordsClickInfo.page
Page number in search results where the ad was shown.

In [250]:
df = pd.read_csv('trafficSource.adwordsClickInfo.page.csv', names = ['index', 'trafficSource.adwordsClickInfo.page']).set_index('index')
df

,trafficSource.adwordsClickInfo.page
index,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1708332,NaN
1708333,NaN
1708334,NaN


In [251]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.adwordsClickInfo.page,value_counts,percent
0,NaN,1633063,95.593727
1,1.0,73913,4.326605
2,2.0,1057,0.061873
3,3.0,172,0.010068
4,4.0,80,0.004683
5,5.0,30,0.001756
6,6.0,10,0.000585
7,7.0,6,0.000351
8,9.0,3,0.000176
9,8.0,1,0.000059


### 8.7 trafficSource.adwordsClickInfo.slot

Position of the Ad. Takes one of the following values:
- RHS
- Top


In [252]:
df = pd.read_csv('trafficSource.adwordsClickInfo.slot.csv', names = ['index', 'trafficSource.adwordsClickInfo.slot']).set_index('index')
df

,trafficSource.adwordsClickInfo.slot
index,
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
1708332,NaN
1708333,NaN
1708334,NaN


In [253]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.adwordsClickInfo.slot,value_counts,percent
0,NaN,1633063,95.593727
1,RHS,42750,2.502434
2,Top,32447,1.899333
3,Google Display Network,77,0.004507


### 8.8 trafficSource.campaign
The campaign value. Usually set by the utm_campaign URL parameter.

In [254]:
df = pd.read_csv('trafficSource.campaign.csv', names = ['index', 'trafficSource.campaign']).set_index('index')
df

,trafficSource.campaign
index,
0,(not set)
1,(not set)
2,(not set)
3,(not set)
4,(not set)
...,...
1708332,(not set)
1708333,(not set)
1708334,(not set)


In [255]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.campaign,value_counts,percent
0,(not set),1604526,93.923272
1,Data Share Promo,32914,1.926669
2,1000557 | GA | US | en | Hybrid | GDN Text+Banner | AS,24410,1.428875
3,1000557 | GA | US | en | Hybrid | GDN Remarketing,15149,0.886769
4,AW - Dynamic Search Ads Whole Site,15146,0.886593
5,AW - Accessories,7972,0.466653
6,Smart Display Campaign,2664,0.155941
7,"""google + redesign/Accessories March 17"" All Users Similar Audiences",1179,0.069014
8,"Page: contains ""/google+redesign/drinkware"" Similar Audiences",611,0.035766
9,"""google + redesign/Accessories March 17"" All Users",562,0.032897


### 8.9 trafficSource.campaignCode (*drop this column*)
Value of the utm_id campaign tracking parameter, used for manual campaign tracking.


In [256]:
df = pd.read_csv('trafficSource.campaignCode.csv', 
                 names = ['index', 'trafficSource.campaignCode']).set_index('index')
df

,trafficSource.campaignCode
index,
100000,NaN
100001,NaN
100002,NaN
100003,NaN
100004,NaN
...,...
199995,NaN
199996,NaN
199997,NaN


In [257]:
vl_count_df = df.value_counts(dropna=False).to_frame()\
                    .rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.campaignCode,value_counts,percent
0,NaN,99999,5.853587
1,11251kjhkvahf,1,0.000059


### 8.10 trafficSource.isTrueDirect
True if the source of the session was Direct (meaning the user typed the name of your website URL into the browser or came to your site via a bookmark), This field will also be true if 2 successive but distinct sessions have exactly the same campaign details. Otherwise NULL.

In [258]:
df = pd.read_csv('trafficSource.isTrueDirect.csv', 
                 names = ['index', 'trafficSource.isTrueDirect']).set_index('index')
df

,trafficSource.isTrueDirect
index,
0,NaN
1,NaN
2,True
3,NaN
4,NaN
...,...
1708332,NaN
1708333,NaN
1708334,NaN


In [259]:
vl_count_df = df.value_counts(dropna=False).to_frame()\
        .rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,trafficSource.isTrueDirect,value_counts
0,NaN,1173819
1,True,534518


### 8.11 trafficSource.keyword
The keyword of the traffic source, usually set when the trafficSource.medium is "organic" or "cpc". Can be set by the utm_term URL parameter.


In [260]:
df = pd.read_csv('trafficSource.keyword.csv', 
                 names = ['index', 'trafficSource.keyword']).set_index('index')
df

,trafficSource.keyword
index,
0,water bottle
1,NaN
2,NaN
3,(not provided)
4,(not provided)
...,...
1708332,NaN
1708333,NaN
1708334,NaN


In [261]:
vl_count_df = df.value_counts(dropna=False).to_frame()\
            .rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.keyword,value_counts,percent
0,NaN,1052780,61.626014
1,(not provided),568933,33.303324
2,(User vertical targeting),25918,1.517148
3,(automatic matching),18464,1.080817
4,6qEhsCssdK0z36ri,10870,0.636291
...,...,...,...
4542,google mechanidise,1,0.000059
4543,google mechendise store,1,0.000059
4544,google men shirt,1,0.000059
4545,google men's,1,0.000059


### 8.12 trafficSource.medium
The medium of the traffic source. Could be "organic", "cpc", "referral", or the value of the utm_medium URL parameter.


In [262]:
df = pd.read_csv('trafficSource.medium.csv', names = ['index', 'trafficSource.medium']).set_index('index')
df

,trafficSource.medium
index,
0,organic
1,referral
2,(none)
3,organic
4,organic
...,...
1708332,referral
1708333,referral
1708334,referral


In [263]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.medium,value_counts,percent
0,organic,591783,34.640882
1,(none),565957,33.129119
2,referral,432963,25.344121
3,cpc,75603,4.425532
4,affiliate,32915,1.926728
5,cpm,8982,0.525774
6,(not set),134,0.007844


### 8.13 trafficSource.referralPath
If trafficSource.medium is "referral", then this is set to the path of the referrer. (The host name of the referrer is in trafficSource.source.)


In [264]:
df = pd.read_csv('trafficSource.referralPath.csv', names = ['index', 'trafficSource.referralPath']).set_index('index')
df

,trafficSource.referralPath
index,
0,NaN
1,/a/google.com/transportation/mtv-services/bikes/bike2workmay2016
2,NaN
3,NaN
4,NaN
...,...
1708332,/yt/about/
1708333,/yt/about/
1708334,/yt/about/ko/


In [265]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.referralPath,value_counts,percent
0,NaN,1142073,66.852910
1,/,138293,8.095183
2,/yt/about/,79163,4.633922
3,/analytics/web/,33112,1.938259
4,/yt/about/tr/,14600,0.854632
...,...,...,...
3192,/intl/eu/yt/about/policies/,1,0.000059
3193,/intl/eu/yt/about/brand-resources/,1,0.000059
3194,/intl/et/yt/about/brand-resources/,1,0.000059
3195,/intl/es_es/permissions/using-the-logo.html,1,0.000059


In [266]:
vl_count_df.groupby('value_counts')['trafficSource.referralPath'].apply(list).to_frame()

trafficSource.referralPath
value_counts                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

### 8.14 trafficSource.source
The source of the traffic source. Could be the name of the search engine, the referring hostname, or a value of the utm_source URL parameter.

In [267]:
df = pd.read_csv('trafficSource.source.csv', names = ['index', 'trafficSource.source']).set_index('index')
df

,trafficSource.source
index,
0,google
1,sites.google.com
2,(direct)
3,google
4,google
...,...
1708332,youtube.com
1708333,youtube.com
1708334,youtube.com


In [268]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,trafficSource.source,value_counts,percent
0,google,658384,38.539469
1,(direct),565975,33.130173
2,youtube.com,329450,19.284837
3,analytics.google.com,37436,2.191371
4,Partners,32931,1.927664
...,...,...,...
340,m.wikihow.com,1,0.000059
341,fr.yhs4.search.yahoo.com,1,0.000059
342,cz.pinterest.com,1,0.000059
343,dailydot.com,1,0.000059


## 9. visitId
An identifier for this session. This is part of the value usually stored as the _utmb cookie. This is only unique to the user. For a completely unique ID, you should use a combination of fullVisitorId and visitId.

In [269]:
df = pd.read_csv('visitId.csv', names = ['index', 'visitId']).set_index('index')
df

,visitId
index,
0,1508198450
1,1508176307
2,1508201613
3,1508169851
4,1508190552
...,...
1708332,1483554750
1708333,1483543798
1708334,1483526434


In [270]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,visitId,value_counts,percent
0,1513125098,28,0.001639
1,1513124981,28,0.001639
2,1513124949,26,0.001522
3,1513124997,24,0.001405
4,1513125008,24,0.001405
...,...,...,...
1665797,1488726052,1,0.000059
1665798,1488726043,1,0.000059
1665799,1488725980,1,0.000059
1665800,1488725740,1,0.000059


## 10. visitNumber 
The session number for this user. If this is the first session, then this is set to 1.

In [271]:
df = pd.read_csv('visitNumber.csv', names = ['index', 'visitNumber']).set_index('index')
df

,visitNumber
index,
0,1
1,6
2,1
3,1
4,1
...,...
1708332,1
1708333,1
1708334,1


In [272]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,visitNumber,value_counts,percent
0,1,1307430,76.532324
1,2,182542,10.685362
2,3,70962,4.153864
3,4,37886,2.217712
4,5,23314,1.364719
...,...,...,...
452,352,1,0.000059
453,403,1,0.000059
454,402,1,0.000059
455,401,1,0.000059


## 11. visitStartTime 
The timestamp (expressed as POSIX time).

In [273]:
df = pd.read_csv('visitStartTime.csv', names = ['index', 'visitStartTime']).set_index('index')
df

,visitStartTime
index,
0,1508198450
1,1508176307
2,1508201613
3,1508169851
4,1508190552
...,...
1708332,1483554750
1708333,1483543798
1708334,1483526434


In [274]:
# TODO: convert to day-month-year hour-minute-second

## 12. CustomDimensions
This section contains any user-level or session-level custom dimensions that are set for a session. This is a repeated field and has an entry for each dimension that is set.

In [275]:
df = pd.read_csv('customDimensions.csv', names = ['index', 'customDimensions']).set_index('index')
df.head()

,customDimensions
index,
0,"[{'index': '4', 'value': 'EMEA'}]"
1,"[{'index': '4', 'value': 'North America'}]"
2,"[{'index': '4', 'value': 'North America'}]"
3,"[{'index': '4', 'value': 'EMEA'}]"
4,"[{'index': '4', 'value': 'Central America'}]"


In [276]:
vl_count_df = df.value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,customDimensions,value_counts,percent
0,"[{'index': '4', 'value': 'North America'}]",768223,44.969055
1,[],333235,19.506397
2,"[{'index': '4', 'value': 'EMEA'}]",313991,18.379922
3,"[{'index': '4', 'value': 'APAC'}]",222071,12.999250
4,"[{'index': '4', 'value': 'South America'}]",45553,2.666511
5,"[{'index': '4', 'value': 'Central America'}]",25264,1.478865


In [277]:
# convert string representation of list to a list
df['customDimensions'] = df['customDimensions'].apply(lambda x: ast.literal_eval(x))

# fill empty string
df['customDimensions'] = df['customDimensions'].apply(lambda x: x[0] if len(x)==1 else "{}")

# convert json string
df = pd.json_normalize(df['customDimensions'])
df.columns = ['customDimensions' + '_' + col for col in df.columns]
df.head()

,customDimensions_index,customDimensions_value
0,4,EMEA
1,4,North America
2,4,North America
3,4,EMEA
4,4,Central America


### 12.1 CustomDimensions Index

In [278]:
vl_count_df = df['customDimensions_index'].to_frame().value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df

,customDimensions_index,value_counts
0,4,1375102
1,NaN,333235


In [279]:
# based on https://support.google.com/analytics/answer/2709828?hl=en#zippy=%2Cin-this-article 
des = {
"4": "Product-level scope"
}
des_df = pd.DataFrame(list(des.items()), columns = ['customDimensions_index', 'description'])

In [280]:
pd.merge(vl_count_df, des_df, on=['customDimensions_index'], how='outer').set_index('customDimensions_index')

,value_counts,description
customDimensions_index,,
4,1375102,Product-level scope
NaN,333235,NaN


### 12.2 CustomDimensions Value

In [281]:
vl_count_df = df['customDimensions_value'].to_frame().value_counts(dropna=False).to_frame().rename(columns={0:'value_counts'}).reset_index()
vl_count_df['percent'] = vl_count_df['value_counts'] / 1708337 * 100
vl_count_df

,customDimensions_value,value_counts,percent
0,North America,768223,44.969055
1,NaN,333235,19.506397
2,EMEA,313991,18.379922
3,APAC,222071,12.999250
4,South America,45553,2.666511
5,Central America,25264,1.478865


In [282]:
des = {
"North America": "North America",
"EMEA": "Europe, the Middle East and Africa",
"APAC": "A-sia, PAC-ific",
"South America": "South America",
"Central America": "Central America"
}
des_df = pd.DataFrame(list(des.items()), columns = ['customDimensions_value', 'description'])

In [283]:
pd.merge(vl_count_df, des_df, on=['customDimensions_value'], how='outer').set_index('customDimensions_value')

,value_counts,percent,description
customDimensions_value,,,
North America,768223,44.969055,North America
NaN,333235,19.506397,NaN
EMEA,313991,18.379922,"Europe, the Middle East and Africa"
APAC,222071,12.999250,"A-sia, PAC-ific"
South America,45553,2.666511,South America
Central America,25264,1.478865,Central America


# FEATURE ENGINEERING

In [284]:
from os import listdir
from datetime import datetime, timedelta
lst_file = listdir()

In [285]:
lst_file = listdir()
df = pd.DataFrame()

for f in lst_file:
    col = f[:-4]
    df[col] = pd.read_csv(f, names = ['index', col],
                             low_memory=False).set_index('index')[col]

- Drop columns

In [286]:
drop_lst = ['trafficSource.campaignCode', 
            'trafficSource.adContent',
            'trafficSource.keyword',
            'trafficSource.adwordsClickInfo.slot',
            'trafficSource.campaign',
            'trafficSource.adwordsClickInfo.page',
            'trafficSource.adwordsClickInfo.gclId',
            'trafficSource.adwordsClickInfo.adNetworkType',
            'totals.transactionRevenue']
# trafficSource.campaignCode
for col in df.columns:
    if len(df[col].value_counts(dropna=False)) == 1:
        drop_lst.append(col)

drop_lst

['trafficSource.campaignCode',
 'trafficSource.adContent',
 'trafficSource.keyword',
 'trafficSource.adwordsClickInfo.slot',
 'trafficSource.campaign',
 'trafficSource.adwordsClickInfo.page',
 'trafficSource.adwordsClickInfo.gclId',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'totals.transactionRevenue',
 'socialEngagementType',
 'device.browserSize',
 'device.screenResolution',
 'device.screenColors',
 'device.operatingSystemVersion',
 'device.mobileDeviceModel',
 'device.mobileInputSelector',
 'device.mobileDeviceMarketingName',
 'device.mobileDeviceInfo',
 'device.mobileDeviceBranding',
 'device.flashVersion',
 'device.language',
 'device.browserVersion',
 'geoNetwork.latitude',
 'geoNetwork.networkLocation',
 'trafficSource.adwordsClickInfo.criteriaParameters',
 'totals.visits',
 'geoNetwork.longitude',
 'geoNetwork.cityId']

In [287]:
df.drop(columns = drop_lst, inplace=True)
df.shape

(1708337, 31)

In [288]:
df["date"] = pd.to_datetime(df["date"], infer_datetime_format=True, format="%Y%m%d")
df['totals.hits'] = df['totals.hits'].astype(float)
df['totals.pageviews'] = df['totals.pageviews'].astype(float)
df['totals.timeOnSite'] = df['totals.timeOnSite'].astype(float)
df['totals.newVisits'] = df['totals.newVisits'].astype(float)
df['totals.transactions'] = df['totals.transactions'].astype(float)
df['device.isMobile'] = df['device.isMobile'].astype(bool)
df['trafficSource.isTrueDirect'] = df['trafficSource.isTrueDirect'].astype(bool)
df['totals.totalTransactionRevenue'] = df['totals.totalTransactionRevenue'].astype(float)
df['totals.totalTransactionRevenue'].fillna(0, inplace=True)

In [289]:
df['customDimensions.value'] = df['customDimensions'].apply(lambda x: ast.literal_eval(x.strip('][') 
                                                            if x != '[]' 
                                                            else '{}').get('value')).to_frame()
df.drop(columns=['customDimensions'], inplace=True)

In [290]:
df.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,geoNetwork.city,totals.bounces,trafficSource.source,trafficSource.referralPath,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.transactions,totals.totalTransactionRevenue,totals.timeOnSite,totals.sessionQualityDim,totals.pageviews,totals.newVisits,totals.hits,geoNetwork.country,geoNetwork.continent,customDimensions.value
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,Organic Search,2017-10-16,3162355547410993243,1508198450,1,1508198450,Firefox,not available in demo dataset,1.0,google,NaN,organic,True,NaN,Windows,False,desktop,not available in demo dataset,(not set),not available in demo dataset,Western Europe,NaN,0.0,NaN,1.0,1.0,1.0,1.0,Germany,Europe,EMEA
1,Referral,2017-10-16,8934116514970143966,1508176307,6,1508176307,Chrome,Cupertino,NaN,sites.google.com,/a/google.com/transportation/mtv-services/bikes/bike2workmay2016,referral,True,NaN,Chrome OS,False,desktop,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,NaN,0.0,28.0,2.0,2.0,NaN,2.0,United States,Americas,North America
2,Direct,2017-10-16,7992466427990357681,1508201613,1,1508201613,Chrome,not available in demo dataset,NaN,(direct),NaN,(none),True,NaN,Android,True,mobile,not available in demo dataset,windjammercable.net,not available in demo dataset,Northern America,NaN,0.0,38.0,1.0,2.0,1.0,2.0,United States,Americas,North America
3,Organic Search,2017-10-16,9075655783635761930,1508169851,1,1508169851,Chrome,not available in demo dataset,NaN,google,NaN,organic,True,NaN,Windows,False,desktop,not available in demo dataset,unknown.unknown,not available in demo dataset,Western Asia,NaN,0.0,1.0,1.0,2.0,1.0,2.0,Turkey,Asia,EMEA
4,Organic Search,2017-10-16,6960673291025684308,1508190552,1,1508190552,Chrome,not available in demo dataset,NaN,google,NaN,organic,True,NaN,Windows,False,desktop,not available in demo dataset,prod-infinitum.com.mx,not available in demo dataset,Central America,NaN,0.0,52.0,1.0,2.0,1.0,2.0,Mexico,Americas,Central America


In [291]:
cnt = df['fullVisitorId'].value_counts().to_frame()
cnt[cnt['fullVisitorId'] == 5]

,fullVisitorId
4170504047351255699,5
9944482549138933048,5
9342641982046386320,5
3863873694540855771,5
1723698628073876286,5
...,...
8200024559872067967,5
9828563986264372577,5
5441713978831708309,5
5354323945861534689,5


In [292]:
df[df['fullVisitorId'] == '8200024559872067967']

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,geoNetwork.city,totals.bounces,trafficSource.source,trafficSource.referralPath,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.transactions,totals.totalTransactionRevenue,totals.timeOnSite,totals.sessionQualityDim,totals.pageviews,totals.newVisits,totals.hits,geoNetwork.country,geoNetwork.continent,customDimensions.value
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
392673,Referral,2017-05-03,8200024559872067967,1493836680,2,1493836680,Chrome,Dublin,NaN,mail.google.com,/mail/u/0/,referral,True,NaN,Linux,False,desktop,(not set),(not set),County Dublin,Northern Europe,NaN,0.0,243.0,NaN,13.0,NaN,17.0,Ireland,Europe,EMEA
831501,Social,2017-05-04,8200024559872067967,1493909382,4,1493909382,Chrome,Dublin,NaN,groups.google.com,/a/google.com/forum/,referral,True,NaN,Linux,False,desktop,(not set),(not set),County Dublin,Northern Europe,NaN,0.0,451.0,NaN,6.0,NaN,7.0,Ireland,Europe,EMEA
831502,Social,2017-05-04,8200024559872067967,1493887993,3,1493887993,Chrome,Dublin,NaN,groups.google.com,/a/google.com/forum/,referral,True,NaN,Linux,False,desktop,(not set),(not set),County Dublin,Northern Europe,NaN,0.0,45.0,NaN,6.0,NaN,7.0,Ireland,Europe,EMEA
831521,Social,2017-05-04,8200024559872067967,1493912932,5,1493912932,Chrome,Dublin,NaN,groups.google.com,/a/google.com/forum/,referral,True,NaN,Linux,False,desktop,(not set),(not set),County Dublin,Northern Europe,1.0,38590000.0,297.0,NaN,14.0,NaN,15.0,Ireland,Europe,EMEA
876486,Referral,2017-02-22,8200024559872067967,1487777294,1,1487777294,Chrome,Dublin,1.0,mail.google.com,/mail/u/0/,referral,True,NaN,Linux,False,desktop,(not set),(not set),County Dublin,Northern Europe,NaN,0.0,NaN,NaN,1.0,1.0,1.0,Ireland,Europe,EMEA


In [293]:
df[df['fullVisitorId'] == '9828563986264372577']

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device.browser,geoNetwork.city,totals.bounces,trafficSource.source,trafficSource.referralPath,trafficSource.medium,trafficSource.isTrueDirect,trafficSource.adwordsClickInfo.isVideoAd,device.operatingSystem,device.isMobile,device.deviceCategory,geoNetwork.metro,geoNetwork.networkDomain,geoNetwork.region,geoNetwork.subContinent,totals.transactions,totals.totalTransactionRevenue,totals.timeOnSite,totals.sessionQualityDim,totals.pageviews,totals.newVisits,totals.hits,geoNetwork.country,geoNetwork.continent,customDimensions.value
index,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1094760,Affiliates,2017-07-01,9828563986264372577,1498917236,2,1498917236,Chrome,not available in demo dataset,1.0,Partners,NaN,affiliate,True,NaN,Windows,False,desktop,not available in demo dataset,impsat.net.ar,not available in demo dataset,South America,NaN,0.0,NaN,NaN,1.0,NaN,1.0,Argentina,Americas,South America
1095176,Organic Search,2017-07-01,9828563986264372577,1498925657,4,1498925657,Chrome,not available in demo dataset,1.0,google,NaN,organic,True,NaN,Windows,False,desktop,not available in demo dataset,telecom.net.ar,not available in demo dataset,South America,NaN,0.0,NaN,NaN,1.0,NaN,1.0,Argentina,Americas,South America
1095427,Organic Search,2017-07-01,9828563986264372577,1498916588,1,1498916588,Chrome,not available in demo dataset,NaN,google,NaN,organic,True,NaN,Windows,False,desktop,not available in demo dataset,telecom.net.ar,not available in demo dataset,South America,NaN,0.0,34.0,NaN,2.0,1.0,2.0,Argentina,Americas,South America
1095807,Organic Search,2017-07-01,9828563986264372577,1498917397,3,1498917397,Chrome,not available in demo dataset,NaN,google,NaN,organic,True,NaN,Windows,False,desktop,not available in demo dataset,telecom.net.ar,not available in demo dataset,South America,NaN,0.0,1843.0,NaN,5.0,NaN,7.0,Argentina,Americas,South America
1440180,Organic Search,2017-07-10,9828563986264372577,1499720791,5,1499720791,Chrome,not available in demo dataset,1.0,google,NaN,organic,True,NaN,Windows,False,desktop,not available in demo dataset,telecom.net.ar,not available in demo dataset,South America,NaN,0.0,NaN,1.0,1.0,NaN,1.0,Argentina,Americas,South America


In [294]:
#replace all empty fields with NaN  
df.replace(['not available in demo dataset', 
            '(not set)'], np.nan, inplace=True)

## Groupby ID

In [321]:
def renamer(desired_name, agg_func):
    def return_func(x):
        return agg_func(x)
    return_func.__name__ = desired_name
    return return_func

In [325]:
def getTimeFramewithFeatures(tr, k=1):
    # train timeframe
    tf = tr.loc[(tr['date'] >= min(tr['date']) + timedelta(days=168*(k-1))) 
              & (tr['date'] < min(tr['date']) + timedelta(days=168*k))]
    # user id in the test timeframe
    tf_fvid = set(tr.loc[(tr['date'] >= min(tr['date']) + timedelta(days=168*k + 46)) 
                       & (tr['date'] < min(tr['date']) + timedelta(days=168*k + 46 + 62))]['fullVisitorId'])
    # user id in the test timeframe appeared in train timeframe
    tf_returned = tf[tf['fullVisitorId'].isin(tf_fvid)]
    # test timeframe
    tf_tst = tr[tr['fullVisitorId'].isin(set(tf_returned['fullVisitorId']))
             & (tr['date'] >= min(tr['date']) + timedelta(days=168*k + 46))
             & (tr['date'] < min(tr['date']) + timedelta(days=168*k + 46 + 62))]
    # for returned user
    tf_target = tf_tst.groupby('fullVisitorId')[['totals.totalTransactionRevenue']]\
                        .sum().apply(np.log1p, axis=1).reset_index()
    tf_target['ret'] = 1
    tf_target.rename(columns={'totals.totalTransactionRevenue': 'target'}, 
                     inplace=True)
    # for new user
    tf_nonret = pd.DataFrame()
    tf_nonret['fullVisitorId'] = list(set(tf['fullVisitorId']) - tf_fvid)    
    tf_nonret['target'] = 0
    tf_nonret['ret'] = 0
    
    tf_target = pd.concat([tf_target, tf_nonret], axis=0).reset_index(drop=True)
    tf_maxdate = max(tf['date'])
    tf_mindate = min(tf['date'])

    tf = tf.groupby('fullVisitorId').agg({          
        'channelGrouping': [renamer('channelGrouping_max', lambda x: x.dropna().max())],
        'visitNumber': [renamer('visitNumber_max', lambda x: x.dropna().max())],
        'device.browser': [renamer('browser_max', lambda x: x.dropna().max())],
        'geoNetwork.city': [renamer('city_max', lambda x: x.dropna().max())],
        'totals.bounces': [renamer('bounces_count', lambda x: x.dropna().count())],
        'trafficSource.source': [renamer('source_max', lambda x: x.dropna().max())],
        'trafficSource.referralPath': [renamer('referralPath_max', lambda x: x.dropna().max())],
        'trafficSource.medium': [renamer('medium_max', lambda x: x.dropna().max())],
        'trafficSource.isTrueDirect': [renamer('isTrueDirect_max', lambda x: x.dropna().max())],
        'trafficSource.adwordsClickInfo.isVideoAd': [renamer('isVideoAd_max', lambda x: x.fillna(True).max())],
        'device.operatingSystem': [renamer('operatingSystem_max', lambda x: x.dropna().max())],
        'device.isMobile': [renamer('isMobile_max', lambda x: x.dropna().max())],
        'device.deviceCategory': [renamer('deviceCategory_max', lambda x: x.dropna().max())],
        'geoNetwork.metro': [renamer('metro_max', lambda x: x.dropna().max())],
        'geoNetwork.networkDomain': [renamer('networkDomain_max',lambda x: x.dropna().max())],
        'geoNetwork.region': [renamer('region_max', lambda x: x.dropna().max())],
        'geoNetwork.subContinent': [renamer('subContinent_max', lambda x: x.dropna().max())],
        'totals.totalTransactionRevenue':  [renamer('totalTransactionRevenue_sum',  lambda x:x.dropna().sum())],
        'totals.transactions' : [renamer('transactions_sum', lambda x:x.dropna().sum())],
        'totals.timeOnSite': [
            renamer('timeOnSite_sum', lambda x: x.dropna().sum()),
            renamer('timeOnSite_min', lambda x: x.dropna().min()),
            renamer('timeOnSite_max', lambda x: x.dropna().max()),
        ],
        'totals.sessionQualityDim': [
            renamer('sessionQualityDim_max', lambda x: x.dropna().max()),
            renamer('sessionQualityDim_min', lambda x: x.dropna().mean()),
            renamer('sessionQualityDim_min', lambda x: x.dropna().min()),
        ],
        'totals.pageviews': [
            renamer('pageviews_sum', lambda x: x.dropna().sum()),
            renamer('pageviews_min', lambda x: x.dropna().min()),
            renamer('pageviews_max', lambda x: x.dropna().max()),
            renamer('pageviews_mean', lambda x: x.dropna().mean()),
        ],
        'totals.newVisits': [renamer('newVisits_max', lambda x: x.fillna(0).max())],
        'totals.hits': [
            renamer('hits_sum', lambda x: x.dropna().sum()),
            renamer('hits_min', lambda x: x.dropna().min()), 
            renamer('hits_max', lambda x: x.dropna().max()), 
            renamer('hits_mean', lambda x: x.dropna().mean()),
        ],
        'geoNetwork.country': [renamer('country_max', lambda x: x.dropna().max())],
        'geoNetwork.continent': [renamer('continent_max', lambda x: x.dropna().max())],
        'customDimensions.value': [renamer('customDimensions.value_max', lambda x: x.dropna().max())],
    })

    tf.columns = tf.columns.droplevel()

    tf = pd.merge(tf, tf_target, left_on='fullVisitorId', right_on='fullVisitorId')
    return tf

In [326]:
print('Get 1st train part')
df1 = getTimeFramewithFeatures(df, k=1)
df1.to_csv('df1.csv')

print('Get 2st train part')
df2 = getTimeFramewithFeatures(df, k=2)
df2.to_csv('df2.csv')

print('Get 3st train part')
df3 = getTimeFramewithFeatures(df, k=3)
df3.to_csv('df3.csv')

print('Get 4st train part')
df4 = getTimeFramewithFeatures(df, k=4)
df4.to_csv('df4.csv')

Get 1st train part
Get 2st train part
Get 3st train part
Get 4st train part


In [327]:
df1

,fullVisitorId,channelGrouping_max,visitNumber_max,browser_max,city_max,bounces_count,source_max,referralPath_max,medium_max,isTrueDirect_max,isVideoAd_max,operatingSystem_max,isMobile_max,deviceCategory_max,metro_max,networkDomain_max,region_max,subContinent_max,totalTransactionRevenue_sum,transactions_sum,timeOnSite_sum,timeOnSite_min,timeOnSite_max,sessionQualityDim_max,sessionQualityDim_min,sessionQualityDim_min,pageviews_sum,pageviews_min,pageviews_max,pageviews_mean,newVisits_max,hits_sum,hits_min,hits_max,hits_mean,country_max,continent_max,customDimensions.value_max,target,ret
0,0000010278554503158,Organic Search,1,Chrome,NaN,0,google,NaN,organic,True,True,Macintosh,False,desktop,NaN,xtra.co.nz,NaN,Australasia,0.0,0.0,194.0,194.0,194.0,NaN,NaN,NaN,8.0,8.0,8.0,8.0,1.0,11.0,11.0,11.0,11.0,New Zealand,Oceania,NaN,0.0,0
1,0000020424342248747,Organic Search,1,Chrome,La Victoria,0,(direct),NaN,(none),True,True,Windows,False,desktop,NaN,munitrujillo.gob.pe,Lima Region,South America,0.0,0.0,297.0,297.0,297.0,NaN,NaN,NaN,13.0,13.0,13.0,13.0,1.0,17.0,17.0,17.0,17.0,Peru,Americas,NaN,0.0,0
2,000005103959234087,Organic Search,1,Chrome,NaN,0,google,NaN,organic,True,True,Android,True,mobile,NaN,comcast.net,NaN,Northern America,0.0,0.0,202.0,202.0,202.0,NaN,NaN,NaN,8.0,8.0,8.0,8.0,1.0,10.0,10.0,10.0,10.0,United States,Americas,North America,0.0,0
3,0000093957001069502,Organic Search,1,Chrome,NaN,0,(direct),NaN,(none),True,True,Windows,False,desktop,NaN,NaN,NaN,Northern America,0.0,0.0,4.0,4.0,4.0,NaN,NaN,NaN,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,United States,Americas,North America,0.0,0
4,0000114156543135683,Social,1,Safari,Istanbul,1,youtube.com,/yt/about/,referral,True,True,Macintosh,False,desktop,NaN,ttnet.com.tr,Istanbul,Western Asia,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Turkey,Asia,EMEA,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
377181,9999761280717362235,Paid Search,2,Safari,NaN,0,(direct),NaN,(none),True,False,iOS,True,tablet,NaN,comcast.net,NaN,Northern America,0.0,0.0,432.0,205.0,227.0,NaN,NaN,NaN,13.0,5.0,8.0,6.5,1.0,16.0,5.0,11.0,8.0,United States,Americas,North America,0.0,0
377182,9999774833256469931,Organic Search,1,Safari,Sydney,1,(direct),NaN,(none),True,True,iOS,True,mobile,NaN,bigair.net.au,New South Wales,Australasia,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,Australia,Oceania,APAC,0.0,0
377183,9999801229043043045,Direct,1,Chrome,NaN,0,(direct),NaN,(none),True,True,Windows,False,desktop,NaN,NaN,NaN,Northern America,0.0,0.0,608.0,608.0,608.0,NaN,NaN,NaN,23.0,23.0,23.0,23.0,1.0,30.0,30.0,30.0,30.0,United States,Americas,North America,0.0,0
377184,9999887420016307570,Organic Search,1,Chrome,NaN,0,(direct),NaN,(none),True,True,Windows,False,desktop,NaN,rogers.com,NaN,Northern America,0.0,0.0,61.0,61.0,61.0,NaN,NaN,NaN,4.0,4.0,4.0,4.0,1.0,7.0,7.0,7.0,7.0,Canada,Americas,North America,0.0,0


## Calculate OUTPUT column

## Drop useless columns

## Handle missing values

# TRAINING MODEL

## Linear Regression

## XGBoost

## LGBM